# `2` European Options
A European option is the type of options contract that allows investors to exercise their options only on the expiration date of that contract.

run `fetch_data.py` to fetch data
<br />
if the data already fetched (`data_old` and `data_new` already available in `data` folder), skip this step

`2.1` Import all the dependencies

In [26]:
from nsepython import *
from datetime import date
import pandas as pd
import numpy as np
from scipy.stats import norm

`2.2` Set `inflation rate` and `bond yield` to calculate `risk free rate` of return for our model
![./images/rfr.png](./images/rfr.png)

In [27]:
# Inflation rate is 3.85% as of 2023 April
inflation_rate = 0.0385

# 10 year bond yield is 7.229% as of 2023 April
bond_yield = 0.07229

# Risk free rate calculation
risk_free_rate = ((1 + bond_yield) /  (1 + inflation_rate)) - 1

print("Risk free rate is: ", risk_free_rate)

Risk free rate is:  0.032537313432835724


`2.3` As the required data is already fetced, read the csv files(`data_old` and `data_new`) from `data` folder and store the data in respective dataframes
<br />

In [28]:

data_old = pd.read_csv('./data/data_old.csv')
data_new = pd.read_csv('./data/data_new.csv')

`2.4` Add new colunm `log_returns` to the dataframe to store log returns
<br />
The log returns are calculated by taking the difference between the log of the `closing price of the current day` and the log of the `closing price of the previous day`.
<br />
`d_std` is the standard deviation of the daily returns
<br />
calculate `annualized_volatility` by multiplying the `daily standard deviation` by the square root of `252` (number of trading days in a year)
<br />
![./images/daily_returns.png](./images/volatility.png)
<br />
`spot_price` is the closing price of the stock on the last trading day of the year

In [29]:
data_old['log_returns'] = np.log(data_old['CH_CLOSING_PRICE'] / data_old['CH_CLOSING_PRICE'].shift(1))
# export to csv
data_old.to_csv(r'./data/data_old_log_returns.csv', index=False)

d_std = data_old['log_returns'].std()
annualized_volatility = d_std * 250 ** 0.5

# Spot price
spot_price = data_old['CH_CLOSING_PRICE'].iloc[-1]

In [30]:
# Using Numpy Arrays instead of Pandas DataFrames
# x = data_old["CH_CLOSING_PRICE"].values
# n = x.shape[0]
# window = int(10*252)
# # Calculating the log returns
# log_returns = np.log(x[n-window:n-1]/x[n-window-1:n-2])

# # Calculating the annualized volatility
# annualized_volatility = np.std(log_returns) * np.sqrt(252)
# spot_price = x[n-1]

# # Print the results
# print(annualized_volatility)
# print(spot_price)

`2.5` Calculate `mean price` of 30 days
<br />
considering the stock follows `brownian motion with drift`

In [31]:
m=10000
T=30.0/250.0
# for t in range(T):
def generate_prices(spot_price, risk_free_rate, annualized_volatility, T):
    expected_price = spot_price * np.exp((risk_free_rate - 0.5 * annualized_volatility ** 2) * T + annualized_volatility * np.sqrt(T) * np.random.normal(0, 1, m))
    return expected_price
mean_price = generate_prices(spot_price, risk_free_rate, annualized_volatility, T)
print(spot_price)
print(mean_price)

1071.15
[1112.87399248 1219.18855512 1158.52709123 ... 1096.19214289 1345.61174594
  942.14854508]


`2.6` Set the `strike price`: the price at which the option can be exercised
<br />
`payoff_generation` function calculates both `call` and `put` options' payoffs
![./images/payoff.png](./images/payoff.png)

In [32]:
strike_price = 1070.00
# Calculating Payoffs
def payoff_generation(strike_price, simulation_count = 10000):
    call_payoffs = []
    put_payoffs = []
    for i in range(simulation_count):
        mean_price = generate_prices(spot_price, risk_free_rate, annualized_volatility, T)
        call_payoff = np.maximum(mean_price - strike_price, 0)
        put_payoff = np.maximum(strike_price - mean_price, 0)
        call_payoffs.append(call_payoff)
        put_payoffs.append(put_payoff)
    return call_payoffs, put_payoffs

`2.7` Declare function to calculate `call` and `put` options' pricing using respective payoffs

In [33]:
# Calculating the call and put option prices
def option_price_calculation(strike_price, simulation_count = 10000):
    call_payoffs, put_payoffs = payoff_generation(strike_price, simulation_count)
    call_price = np.mean(call_payoffs) * np.exp(-risk_free_rate * T)
    put_price = np.mean(put_payoffs) * np.exp(-risk_free_rate * T)
    return call_payoffs, put_payoffs, call_price, put_price

`2.8` Call above declared function to calculate `call` and `put` pricing

In [34]:
# Executing the function
call_payoffs, put_payoffs, call_price, put_price = option_price_calculation(strike_price)

In [35]:
# Printing the results
print("Call Option price is: ", call_price)
print("Put Option price is: ", put_price)

Call Option price is:  80.30291890028367
Put Option price is:  74.99167181650449


In [36]:
# Calculating Put Call Parity
def put_call_parity_calculation(call_price, put_price, spot_price, strike_price, risk_free_rate, T):
    lhs = call_price - put_price
    rhs = spot_price - strike_price * np.exp(-risk_free_rate * T)
    return round(lhs) == round(rhs)

# Executing the function
put_call_parity_calculation(call_price, put_price, spot_price, strike_price, risk_free_rate, T)

True

In [37]:
# Now lets calculate the option price using Black Scholes formula
# First we need to calculate the d1 and d2 values

# Calculating d1
def d1_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d1 = (np.log(spot_price/strike_price) + (risk_free_rate + 0.5 * annualized_volatility**2) * T) / (annualized_volatility * np.sqrt(T))
    return d1

# Calculating d2
def d2_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d2 = (np.log(spot_price/strike_price) + (risk_free_rate - 0.5 * annualized_volatility**2) * T) / (annualized_volatility * np.sqrt(T))
    return d2

In [38]:
# Calculating the call and put option prices using Black Scholes formula
def option_price_calculation_black_scholes(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d1 = d1_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
    d2 = d2_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
    call_price = spot_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2)
    put_price = strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2) - spot_price * norm.cdf(-d1)
    return call_price, put_price

In [39]:
# Executing the function
call_price_black_scholes, put_price_black_scholes = option_price_calculation_black_scholes(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
print("Call Option price using Black Scholes formula is: ", call_price_black_scholes)
print("Put Option price using Black Scholes formula is: ", put_price_black_scholes)

Call Option price using Black Scholes formula is:  80.30715965784196
Put Option price using Black Scholes formula is:  74.98751405418727


In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA


In [60]:
data = pd.read_csv('./data/data_old.csv')
data = data.set_index('CH_TIMESTAMP')
data.index = pd.to_datetime(data.index)
data = data.sort_index()
data['CH_CLOSING'] = data['CH_CLOSING_PRICE'].pct_change()


In [61]:
data

,_id,CH_SYMBOL,CH_SERIES,CH_MARKET_TYPE,CH_TRADE_HIGH_PRICE,CH_TRADE_LOW_PRICE,CH_OPENING_PRICE,CH_CLOSING_PRICE,CH_LAST_TRADED_PRICE,CH_PREVIOUS_CLS_PRICE,...,CH_ISIN,TIMESTAMP,createdAt,updatedAt,__v,SLBMH_TOT_VAL,VWAP,mTIMESTAMP,CA,CH_CLOSING
CH_TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2002-01-01,641f31b44708d50007274d25,HDFCBANK,EQ,N,230.00,222.75,228.00,225.05,226.00,225.60,...,NaN,2001-12-31T18:30:00.000Z,2023-03-25T17:39:00.692Z,2023-03-25T17:39:00.692Z,0,NaN,225.02,01-Jan-2002,NaN,NaN
2002-01-02,641f32744438f60007b7ce4d,HDFCBANK,EQ,N,225.75,223.00,224.00,225.00,225.00,225.05,...,NaN,2002-01-01T18:30:00.000Z,2023-03-25T17:42:12.909Z,2023-03-25T17:42:12.909Z,0,NaN,224.53,02-Jan-2002,NaN,-0.000222
2002-01-03,641f3270adde0b0007d23e28,HDFCBANK,EQ,N,227.95,222.20,225.00,226.25,227.75,225.00,...,NaN,2002-01-02T18:30:00.000Z,2023-03-25T17:42:08.208Z,2023-03-25T17:42:08.208Z,0,NaN,224.70,03-Jan-2002,NaN,0.005556
2002-01-04,641f326a341dfa0007d37446,HDFCBANK,EQ,N,228.00,223.50,225.00,223.95,223.75,226.25,...,NaN,2002-01-03T18:30:00.000Z,2023-03-25T17:42:02.801Z,2023-03-25T17:42:02.801Z,0,NaN,224.49,04-Jan-2002,NaN,-0.010166
2002-01-07,641f31364708d500072748bd,HDFCBANK,EQ,N,226.35,221.55,224.00,223.95,224.15,223.95,...,NaN,2002-01-06T18:30:00.000Z,2023-03-25T17:36:54.390Z,2023-03-25T17:36:54.390Z,0,NaN,223.79,07-Jan-2002,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-03-28,641ebc1f4438f60007b5a3a4,HDFCBANK,EQ,N,1053.00,1041.65,1045.00,1047.50,1052.30,1049.35,...,INE040A01026,2016-03-27T18:30:00.000Z,2023-03-25T09:17:19.098Z,2023-03-25T09:17:19.098Z,0,NaN,1048.62,28-Mar-2016,NaN,-0.001763
2016-03-28,641ebc1f4438f60007b5a3a4,HDFCBANK,EQ,N,1053.00,1041.65,1045.00,1047.50,1052.30,1049.35,...,INE040A01026,2016-03-27T18:30:00.000Z,2023-03-25T09:17:19.098Z,2023-03-25T09:17:19.098Z,0,NaN,1048.62,28-Mar-2016,NaN,0.000000
2016-03-29,641ebc2931da23000791a595,HDFCBANK,EQ,N,1059.50,1044.25,1047.50,1053.80,1053.00,1047.50,...,INE040A01026,2016-03-28T18:30:00.000Z,2023-03-25T09:17:29.178Z,2023-03-25T09:17:29.178Z,0,NaN,1054.95,29-Mar-2016,NaN,0.006014


In [62]:
train = data['CH_CLOSING'].iloc[:-100]
test = data['CH_CLOSING'].iloc[-100:]

In [63]:
test

CH_TIMESTAMP
2015-11-09   -0.014895
2015-11-10   -0.009579
2015-11-11   -0.000190
2015-11-13   -0.002229
2015-11-16    0.008460
                ...   
2016-03-28   -0.001763
2016-03-28    0.000000
2016-03-29    0.006014
2016-03-30    0.010581
2016-03-31    0.005822
Name: CH_CLOSING, Length: 100, dtype: float64

In [146]:
model = ARIMA(train, order=(1, 0, 0))

model_fit = model.fit()

/Users/ayushdattajaiswal/miniforge3/envs/mlp/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ayushdattajaiswal/miniforge3/envs/mlp/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ayushdattajaiswal/miniforge3/envs/mlp/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [147]:
predict = model_fit.predict(len(train), len(train)+len(test)-1, typ='levels')

/Users/ayushdattajaiswal/miniforge3/envs/mlp/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [148]:
predict

3536    0.000875
3537    0.000875
3538    0.000875
3539    0.000875
3540    0.000875
          ...   
3631    0.000875
3632    0.000875
3633    0.000875
3634    0.000875
3635    0.000875
Name: predicted_mean, Length: 100, dtype: float64

In [149]:
# Now Apply percentage change to the predicted values
predict = (predict + 1)
predict


3536    1.000875
3537    1.000875
3538    1.000875
3539    1.000875
3540    1.000875
          ...   
3631    1.000875
3632    1.000875
3633    1.000875
3634    1.000875
3635    1.000875
Name: predicted_mean, Length: 100, dtype: float64

In [150]:
yo=np.zeros(predict.shape)
yo[0] = predict[3536]
for i in range(3537, 3636):
    yo[i-3536] = yo[i-3537] * predict[i]

In [151]:
yo

array([1.00087463, 1.00175048, 1.00262709, 1.00350446, 1.00438261,
       1.00526152, 1.0061412 , 1.00702165, 1.00790288, 1.00878487,
       1.00966764, 1.01055117, 1.01143548, 1.01232057, 1.01320643,
       1.01409306, 1.01498047, 1.01586866, 1.01675763, 1.01764737,
       1.01853789, 1.01942919, 1.02032127, 1.02121413, 1.02210777,
       1.02300219, 1.0238974 , 1.02479339, 1.02569016, 1.02658772,
       1.02748607, 1.0283852 , 1.02928512, 1.03018582, 1.03108731,
       1.03198959, 1.03289267, 1.03379653, 1.03470118, 1.03560662,
       1.03651286, 1.03741989, 1.03832771, 1.03923633, 1.04014574,
       1.04105595, 1.04196696, 1.04287876, 1.04379136, 1.04470476,
       1.04561896, 1.04653395, 1.04744975, 1.04836635, 1.04928375,
       1.05020196, 1.05112097, 1.05204078, 1.0529614 , 1.05388282,
       1.05480505, 1.05572809, 1.05665193, 1.05757658, 1.05850205,
       1.05942832, 1.0603554 , 1.06128329, 1.062212  , 1.06314152,
       1.06407185, 1.065003  , 1.06593496, 1.06686773, 1.06780

In [152]:
data['CH_CLOSING_PRICE'][-100]

1064.8

In [153]:
prices=yo*data['CH_CLOSING_PRICE'][-100]

In [154]:
prices

array([1065.73130781, 1066.66390788, 1067.59732185, 1068.53155264,
       1069.46660095, 1070.4024675 , 1071.33915301, 1072.2766582 ,
       1073.21498377, 1074.15413045, 1075.09409896, 1076.03489001,
       1076.97650433, 1077.91894264, 1078.86220565, 1079.80629409,
       1080.75120868, 1081.69695014, 1082.6435192 , 1083.59091659,
       1084.53914302, 1085.48819922, 1086.43808592, 1087.38880385,
       1088.34035372, 1089.29273628, 1090.24595225, 1091.20000235,
       1092.15488732, 1093.1106079 , 1094.0671648 , 1095.02455876,
       1095.98279052, 1096.9418608 , 1097.90177035, 1098.86251989,
       1099.82411016, 1100.7865419 , 1101.74981584, 1102.71393272,
       1103.67889328, 1104.64469826, 1105.61134839, 1106.57884441,
       1107.54718707, 1108.5163771 , 1109.48641525, 1110.45730225,
       1111.42903886, 1112.40162582, 1113.37506386, 1114.34935374,
       1115.3244962 , 1116.30049198, 1117.27734184, 1118.25504651,
       1119.23360676, 1120.21302332, 1121.19329694, 1122.17442

In [139]:
yo2 = data['CH_CLOSING_PRICE'][-100:].to_numpy()

In [140]:
yo2

array([1064.8 , 1054.6 , 1054.4 , 1052.05, 1060.95, 1060.8 , 1048.6 ,
       1068.9 , 1070.85, 1066.05, 1064.35, 1063.95, 1079.4 , 1077.75,
       1083.  , 1077.8 , 1076.75, 1058.9 , 1061.95, 1049.25, 1047.45,
       1060.6 , 1046.35, 1055.05, 1059.45, 1067.3 , 1080.25, 1073.  ,
       1075.4 , 1066.45, 1074.1 , 1074.  , 1077.25, 1077.95, 1074.3 ,
       1082.15, 1088.75, 1070.5 , 1062.4 , 1067.1 , 1056.2 , 1062.35,
       1062.35, 1058.6 , 1046.95, 1060.15, 1049.75, 1042.15, 1025.85,
       1035.8 , 1018.3 , 1023.  , 1030.3 , 1041.15, 1035.65, 1031.7 ,
       1049.85, 1059.75, 1053.8 , 1043.85, 1049.2 , 1055.25, 1035.85,
       1026.05, 1012.95,  975.3 ,  971.4 ,  973.6 ,  973.45,  974.9 ,
        974.9 ,  989.45,  989.3 ,  988.75,  970.05,  949.6 ,  942.65,
        961.  ,  971.85,  984.15, 1007.2 , 1015.85, 1020.55, 1014.15,
       1024.8 , 1021.25, 1028.7 , 1027.55, 1026.7 , 1030.45, 1018.4 ,
       1028.4 , 1045.6 , 1053.85, 1049.35, 1047.5 , 1047.5 , 1053.8 ,
       1064.95, 1071

In [141]:
yo2.shape

(100,)